# Take a look at PER1 BAC STARR-seq

The goal is to understand the PER1 BAC STARR-seq data. This is a good dataset to test 
- (1) the fragment model 
- (2) the fragment simulation

## Going to the directories

In [15]:
ls /data/reddylab/gjohnson/STARRseq_projects

20220812_GDJ_HepG2_WGSS_redo_bcl_convert  NatCom_manuscript_analyses
fragment_orientation                      RD_work
GR_agonists_study                         splitSTARR
junk


In [16]:
ls /data/reddylab/gjohnson/STARRseq_projects/RD_work

ATAC_test
BAC_ecoli_background_and_insert_size_selection
Comp_Cell_Comparison
Emulsion_PCR_Test
Gyrase_test
hg19
hybridizing_1kb_frags_to_flowcell
miseq_QC_0.45x_insert_length
ORI_and_SCP_PER1_BAC_Lib
plasmid_dose_response
Polymerase_test
Retransform_test
Robot_test
Stark_WGSS_HeLA
UMIs
Undertermined_reads_PER1_BAC_with_SJC_BAC_library


In [17]:
ls /data/reddylab/gjohnson/STARRseq_projects/RD_work/ORI_and_SCP_PER1_BAC_Lib

alignments                                   ORI_specific_dex_peaks.bed
bigwigs                                      peak_fastas
counts                                       plots
delme2                                       pooled_library_input_data
fastqc                                       positive_control.bed
fastqs                                       preseq.complexity
lengths                                      preseq_expected_distinct.txt
macs_analysis                                ref.fa.bfile
ORI_PER1_BACSS_Dex1.tmp.bam                  ref_output
ORI_PER1_BACSS_Dex1.tmp.bam.bai              SCP_PER1_BACSS_Dex1.tmp.bam
ORI_PER1_BACSS_Dex2.tmp.bam                  SCP_PER1_BACSS_Dex2.tmp.bam
ORI_PER1_BACSS_EtOH1.tmp.bam                 SCP_PER1_BACSS_EtOH1.tmp.bam
ORI_PER1_BACSS_EtOH2.tmp.bam                 SCP_PER1_BACSS_EtOH2.tmp.bam
ORI_specific_dex_peaks_absent_from_EtOH.bed  scripts


In [22]:
ls /data/reddylab/gjohnson/STARRseq_projects/RD_work/ORI_and_SCP_PER1_BAC_Lib/scripts

align.sh                   inputs              PER1_BAC.fa
coverage_in_peaks_plots.R  macs_peak_calls.sh  plot_fragment_lengths.R
coverage_in_peaks.sh       ncis.2.R            prep_2bit_file.sh
dedup_inputs               PER1_BAC.2bit
dedup_UMIs.sh              PER1_BAC.bed


In [23]:
cat /data/reddylab/gjohnson/STARRseq_projects/RD_work/ORI_and_SCP_PER1_BAC_Lib/scripts/align.sh

#!/bin/bash
#
#SBATCH --mem=100000
#SBATCH -J align
#SBATCH -o align.%J.output
#SBATCH -e align.%J.error
#SBATCH --array=1-8
#SBATCH --cpus-per-task=22
#SBATCH --mail-user=graham.johnson@duke.edu
#SBATCH --mail-type=FAIL,END
#SBATCH -p new

#LOAD SOFTWARE
export PATH="/data/reddylab/gjohnson/preseq/:$PATH"

module load bowtie2
module load samtools
module load bedtools2
module load R/3.2.0-gcb01

#SET VARIABLES
R1=$(head -${SLURM_ARRAY_TASK_ID} scripts/inputs | tail -1 | awk '{print$1}')
R2=$(head -${SLURM_ARRAY_TASK_ID} scripts/inputs | tail -1 | awk '{print$2}')
SAMPLE=$(head -${SLURM_ARRAY_TASK_ID} scripts/inputs | tail -1 | awk '{print$3}')


#RUN FASTQC 
/data/reddylab/software/FastQC/fastqc ${R1} ${R2} \
-t 22 \
-o fastqc/


#MAKE SAMPLE DIRECTORY
mkdir alignments/${SAMPLE}


#ALIGN SAMPLES
bowtie2 \
-x /data/reddylab/gjohnson/reference_data/hg38 \
-p 31 \
-X 2000 \
--sensitive \
--un alignments/${SAMPLE}/${SAMPLE}.unmapped.sam \
-1 ${R1} \
-2 ${R2} \
| samtools view -@ 22 -Shu -L

In [20]:
head -1 /data/reddylab/gjohnson/STARRseq_projects/RD_work/ORI_and_SCP_PER1_BAC_Lib/scripts/PER1_BAC.fa

>chr17:7981103-8192310


<font color='salmon'> **Questions** </font>
- <font color='salmon'> Will the folder be in the same directory since you have left the lab? </font>
- <font color='salmon'> just curious: what does RD_work stands for? </font>

**Note**

## Alignment files

In [4]:
FDIRY=/data/reddylab/gjohnson/STARRseq_projects/RD_work/ORI_and_SCP_PER1_BAC_Lib/alignments/
ls -1 ${FDIRY}

bed_filtered_deduped_input_reads.bam
bed_filtered_deduped_input_reads.bam.bai
bed_filtered_deduped_input_reads_chr1.bam
bed_filtered_deduped_input_reads_chr1.sam
bed_filtered_deduped_input_reads.sort.bam
bed_filtered_deduped_input_reads.sort.bam.bai
ORI_PER1_BACSS_Dex1
ORI_PER1_BACSS_Dex2
ORI_PER1_BACSS_EtOH1
ORI_PER1_BACSS_EtOH2
SCP_PER1_BACSS_Dex1
SCP_PER1_BACSS_Dex2
SCP_PER1_BACSS_EtOH1
SCP_PER1_BACSS_EtOH2


```
I would recommend only using alignments with 'ORI' in the name. 'ORI' and 'SCP' refer to different plasmid vector systems. 'ORI' is the most up to date and is what I have used for all of my more recent experiments, including the whole genome STARR-seq data you have been working on.
```

### Look into the ORI files

**Condition: EtOH**

In [5]:
ls -1 ${FDIRY}/ORI_PER1_BACSS_EtOH1

ORI_PER1_BACSS_EtOH1_all_alignments.bedpe
ORI_PER1_BACSS_EtOH1.dedup.bam
ORI_PER1_BACSS_EtOH1.dedup.bam.bai
ORI_PER1_BACSS_EtOH1.dedup.bedpe
ORI_PER1_BACSS_EtOH1.f3q10.bam
ORI_PER1_BACSS_EtOH1.f3q10.bam.bai
ORI_PER1_BACSS_EtOH1_markdup_stats.txt
ORI_PER1_BACSS_EtOH1.UMI_tagged.bai
ORI_PER1_BACSS_EtOH1.UMI_tagged.bam


In [6]:
ls -1 ${FDIRY}/ORI_PER1_BACSS_EtOH2

ORI_PER1_BACSS_EtOH2_all_alignments.bedpe
ORI_PER1_BACSS_EtOH2.dedup.bam
ORI_PER1_BACSS_EtOH2.dedup.bam.bai
ORI_PER1_BACSS_EtOH2.dedup.bedpe
ORI_PER1_BACSS_EtOH2.f3q10.bam
ORI_PER1_BACSS_EtOH2.f3q10.bam.bai
ORI_PER1_BACSS_EtOH2_markdup_stats.txt
ORI_PER1_BACSS_EtOH2.UMI_tagged.bai
ORI_PER1_BACSS_EtOH2.UMI_tagged.bam


In [7]:
head -10 ${FDIRY}/ORI_PER1_BACSS_EtOH1/ORI_PER1_BACSS_EtOH1_all_alignments.bedpe

chr1	23733039	23734785
chr1	23733039	23734785
chr1	23839539	23839643
chr1	23839539	23839643
chr1	26925066	26925955
chr1	28495644	28497353
chr1	75853079	75853260
chr1	183913368	183913460
chr10	12195941	12196025
chr10	12195941	12196025


In [8]:
head -10 ${FDIRY}/ORI_PER1_BACSS_EtOH1/ORI_PER1_BACSS_EtOH1.dedup.bedpe

chr1	23733039	23734785
chr1	23839539	23839643
chr1	26925066	26925955
chr1	28495644	28497353
chr1	75853079	75853260
chr1	183913368	183913460
chr10	12195941	12196025
chr10	68688532	68688715
chr10	95608940	95609140
chr10	126868805	126869010


In [10]:
tail -10 ${FDIRY}/ORI_PER1_BACSS_EtOH1/ORI_PER1_BACSS_EtOH1.dedup.bedpe

chr8	144298481	144299695
chr9	6760585	6761248
chr9	88044114	88044318
chr9	116794141	116794281
chr9	121307597	121307869
chr9	121307597	121307869
chrX	23829274	23831031
chrX	116038896	116039172
chrX	133862432	133862642
chrY	13928965	13929179


In [12]:
awk -v OFS='\t' '{print $1}' < ${FDIRY}/ORI_PER1_BACSS_EtOH1/ORI_PER1_BACSS_EtOH1.dedup.bedpe | sort | uniq -c

      6 chr1
      4 chr10
      2 chr11
      8 chr12
      4 chr13
      4 chr14
      2 chr15
      5 chr16
  16678 chr17
      3 chr18
     10 chr19
      7 chr2
      3 chr20
      1 chr21
      2 chr22
      3 chr3
      2 chr4
      1 chr5
      4 chr6
      5 chr7
      4 chr8
      5 chr9
      3 chrX
      1 chrY


**Condition: Dex**

In [9]:
ls -1 ${FDIRY}/ORI_PER1_BACSS_Dex1

ORI_PER1_BACSS_Dex1_all_alignments.bedpe
ORI_PER1_BACSS_Dex1.dedup.bam
ORI_PER1_BACSS_Dex1.dedup.bam.bai
ORI_PER1_BACSS_Dex1.dedup.bedpe
ORI_PER1_BACSS_Dex1.f3q10.bam
ORI_PER1_BACSS_Dex1.f3q10.bam.bai
ORI_PER1_BACSS_Dex1_markdup_stats.txt
ORI_PER1_BACSS_Dex1.UMI_tagged.bai
ORI_PER1_BACSS_Dex1.UMI_tagged.bam


<font color='salmon'> **Questions** </font>
- <font color='salmon'> Is it correct that the dedup is the deduplication of all alignments? </font>
- <font color='salmon'> I remember you mentioned that the deduplication does not have much effect. Is it hold in PER1 BAC data as well? </font>
- <font color='salmon'> Is it correct that PER1 BAC STARR-seq is using BAC genome that covers PER1 region as the input library? Why are there chromosomes other than chr17?</font>

**Note**